In [12]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


# Function to save data to CSV file
def save_to_csv(data, filename):
    if data:
        keys = data[0].keys()
        with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=keys)
            writer.writeheader()
            writer.writerows(data)
    else:
        print("No data to save.")


# Start the browserhcol w44
service = Service(executable_path='/snap/bin/geckodriver')
browser = webdriver.Firefox(service=service)


def extract_data(browser):
    data_list = []
    try:
        parent_div = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "list-mainarea")))
        
        project_wrappers = parent_div.find_elements(By.CLASS_NAME, "cardholder")
        for project_wrapper in project_wrappers:
            try:
                project_name = project_wrapper.find_element(By.CLASS_NAME, "seller-info").text
                project_price = project_wrapper.find_element(By.CLASS_NAME, "price").text
                builtup_area = project_wrapper.find_element(By.CLASS_NAME, "size").text
                bhk = project_wrapper.find_element(By.CLASS_NAME, "val").text
                address = project_wrapper.find_element(By.CLASS_NAME, "loclink").text
                 
                    
                # Extracting additional details
                details_ul = project_wrapper.find_element(By.CLASS_NAME, "listing-details")
                details_li = details_ul.find_elements(By.CLASS_NAME, "keypoint")

                deposit = None
                bathrooms = None
                facing = None

                for li in details_li:
                    title = li.get_attribute("title")
                    if title == "deposit":
                        deposit = li.text.strip()
                    elif title == "bathrooms":
                        bathrooms = li.find_element(By.TAG_NAME, "span").text
                    elif title == "facing":
                        facing = li.text.strip()

                # Adding all details to the data list
                data_list.append({
                    "Project Name": project_name,
                    "Project Price": project_price,
                    "Builtup Area": builtup_area,
                    "BHK": bhk,
                    "Deposit": deposit,
                    "Bathrooms": bathrooms,
                    "Facing": facing,
                    "Address" : address,
                    "Status" : informations
                })
            except Exception as e:
                print(f"Error extracting data for a project: {e}")
    except Exception as e:
        print(f"Error scraping data: {e}")
    return data_list


base_url = 'https://www.makaan.com/listings?listingType=rent&pageType=CITY_URLS&cityName=Mumbai&cityId=18&templateId=MAKAAN_CITY_LISTING_BUY&page='
page_count = 1
all_data = []

# Loop through each page
for page in range(1, page_count + 1):
    url = base_url + str(page)
    browser.get(url)
    # Add a delay to ensure page loads completely
    time.sleep(5)
    page_data = extract_data(browser)
    print(f"Page {page} Data:")
    for project in page_data:
        print(project)
    all_data.extend(page_data)

# Close the browser properly
browser.quit()

# Write all_data to CSV file
save_to_csv(all_data, 'data5.csv')

print("-- done --")

Page 1 Data:
{'Project Name': 'Kasturi Developers\nBUILDER\n-', 'Project Price': '20,000', 'Builtup Area': '1132', 'BHK': '2', 'Deposit': 'No Deposit', 'Bathrooms': '2 bathrooms', 'Facing': 'NorthEast facing', 'Address': 'Ulwe, Mumbai', 'Status': [<selenium.webdriver.remote.webelement.WebElement (session="a91ae547-845e-4a2e-951c-b54343d02421", element="115cb268-b040-4cd3-96a0-baa88a15498e")>]}
{'Project Name': 'Kasturi Developers\nBUILDER\n-', 'Project Price': '36,000', 'Builtup Area': '1668', 'BHK': '3', 'Deposit': 'No Deposit', 'Bathrooms': '3 bathrooms', 'Facing': 'NorthEast facing', 'Address': 'Ulwe, Mumbai', 'Status': [<selenium.webdriver.remote.webelement.WebElement (session="a91ae547-845e-4a2e-951c-b54343d02421", element="55c886ce-1c80-4121-ae60-15450c0f9857")>]}
{'Project Name': 'Kasturi Developers\nBUILDER\n-', 'Project Price': '35,000', 'Builtup Area': '1700', 'BHK': '3', 'Deposit': 'No Deposit', 'Bathrooms': '3 bathrooms', 'Facing': 'NorthEast facing', 'Address': 'Ulwe, Mumb